In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

In [79]:
# Connect to the database
connexion_deli = sqlite3.connect("Databases/deliveroo.db")
cursor_deli = connexion_deli.cursor()

# Execute query
cursor_deli.execute("""
SELECT 
    category,
    AVG(rating) as average_rating,
    'Deliveroo'
FROM 
    restaurants
WHERE 
    category IN ('Italian', 'Burgers', 'Sushi', 'Turkish', 'Indian', 'Thai')
GROUP BY category

""")
results_deli = cursor_deli.fetchall()

connexion_deli.close()

print(results_deli)

[('Burgers', 2.7536423841059605, 'Deliveroo'), ('Indian', 3.0, 'Deliveroo'), ('Italian', 2.8982456140350874, 'Deliveroo'), ('Sushi', 2.6983739837398373, 'Deliveroo'), ('Thai', 3.035185185185185, 'Deliveroo'), ('Turkish', 3.0051724137931037, 'Deliveroo')]


In [81]:
df_deli = pd.DataFrame(results_deli, columns=['Category', 'Average rating', 'Platform'])
df_deli.head(6)

,Category,Average rating,Platform
0,Burgers,2.753642,Deliveroo
1,Indian,3.000000,Deliveroo
2,Italian,2.898246,Deliveroo
3,Sushi,2.698374,Deliveroo
4,Thai,3.035185,Deliveroo
5,Turkish,3.005172,Deliveroo


In [73]:
# Connect to the database
connexion_take = sqlite3.connect("Databases/takeaway.db")
cursor_take = connexion_take.cursor()

# Execute query
cursor_take.execute("""
SELECT 
    CASE
        WHEN categories_restaurants.category_id = 'italian-pizza_271' THEN 'Italian'
        WHEN categories_restaurants.category_id = 'pasta_1614' THEN 'Italian'
        WHEN categories_restaurants.category_id = 'italian-pizza_271' THEN 'Italian'
        WHEN categories_restaurants.category_id = 'burgers_1984' THEN 'Burgers'
        WHEN categories_restaurants.category_id = 'curry_731,' THEN 'Indian'
        WHEN categories_restaurants.category_id = 'indian_71' THEN 'Indian'
        WHEN categories_restaurants.category_id = 'japanene_551' THEN 'Sushi'
        WHEN categories_restaurants.category_id = 'pita-kebab_651' THEN 'Turkish'
        WHEN categories_restaurants.category_id = 'shoarma_2014' THEN 'Turkish'
        WHEN categories_restaurants.category_id = 'sushi_51' THEN 'Sushi'
        WHEN categories_restaurants.category_id = 'thai_61' THEN 'Thai'
        WHEN categories_restaurants.category_id = 'turkish-pizza' THEN 'Turkish'
        WHEN categories_restaurants.category_id = 'turkish_511' THEN 'Turkish'
        ELSE categories_restaurants.category_id
    END AS category,
    AVG(restaurants.ratings) AS average_rating,
    'Takeaway'
FROM 
    categories_restaurants
INNER JOIN  
    restaurants
ON
    categories_restaurants.restaurant_id = restaurants.primarySlug
WHERE 
    category IN ('Italian', 'Burgers', 'Sushi', 'Turkish', 'Indian', 'Thai')
GROUP BY 
    category
ORDER BY 
    COUNT(categories_restaurants.category_id) DESC;
""")
results_take = cursor_take.fetchall()

connexion_take.close()

print(results_take)


[('Italian', 3.7786774628879893, 'Takeaway'), ('Turkish', 3.857950191570881, 'Takeaway'), ('Burgers', 3.6394662921348315, 'Takeaway'), ('Sushi', 4.013, 'Takeaway'), ('Thai', 4.0196969696969695, 'Takeaway'), ('Indian', 3.5328125, 'Takeaway')]


In [82]:
# Story querry results in a dataframe
df_take = pd.DataFrame(results_take, columns=['Category', 'Average rating', 'Platform'])
df_take.head(6)

,Category,Average rating,Platform
0,Italian,3.778677,Takeaway
1,Turkish,3.857950,Takeaway
2,Burgers,3.639466,Takeaway
3,Sushi,4.013000,Takeaway
4,Thai,4.019697,Takeaway
5,Indian,3.532812,Takeaway


In [ ]:
# Connect to the database
connexion_uber = sqlite3.connect("Databases/ubereats.db")
cursor_uber = connexion_uber.cursor()

# Execute querry
cursor_uber.execute("""
SELECT 
    ,
    'Ubereats' 
FROM restaurants
INNER JOIN 
    menu_items 
ON 
    restaurants.id = menu_items.restaurant_id
WHERE 
    restaurants.rating__rating_value > 0.1 
GROUP BY 
    menu_items.restaurant_id
HAVING 
    AVG(menu_items.price) / 100 > 10     -- menu_items.price is in eurocent. Filter by average price > 10 to filter out sandwich shops and strage values
ORDER BY 
    price_to_rating_ratio ASC; 
    
""")
results_uber = cursor_uber.fetchall()

connexion_uber.close()

print(results_uber)